In [1]:
import pandas as pd
import numpy as np
'import tensorflow as tf



2021-07-26 18:44:12.263940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-26 18:44:12.264027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
data_path = "./data/"

train_id = pd.read_csv(data_path + "train_identity.csv") 
train_tra = pd.read_csv(data_path + "train_transaction.csv")

In [ ]:

test_id = pd.read_csv(data_path + "test_identity.csv")
test_tra = pd.read_csv(data_path + "test_transaction.csv")

In [ ]:
test_id.head()

In [ ]:
test_tra.head()